In [27]:
import pandas as pd
import pathlib

number_of_samples = 10

# data_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G200/cases/000/data/cp_t.hdf")
# output_path = data_path.parent / "cp_t.h5"

# data_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G200/cases/000/data/peak_static.h5")
# output_path = data_path.parent / "static_pressure.h5"

data_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G200/cases/000/data/bodies.G200.merged.data.h5")
output_path = data_path.parent / "body_data.h5"

original_df = pd.read_hdf(data_path)

with pd.HDFStore(output_path,  mode='w') as store:
    step = original_df.time_step.nunique() //  number_of_samples
    time_arr = original_df.time_step.unique()

    for i in range(number_of_samples):
        df = original_df.loc[(original_df.time_step >= time_arr[i * step]) & (original_df.time_step <= time_arr[(i + 1) * step - 1])].copy()
        print(df.time_step.min(), df.time_step.max(), i)
        store.put(f'range_{i}', df)

    if time_arr[(i + 1) * step - 1] != time_arr[-1]:
        df = original_df.loc[(original_df.time_step >= time_arr[(i + 1) * step]) & (original_df.time_step <= time_arr[-1])].copy()
        print(df.time_step.min(), df.time_step.max(),i + 1)
        store.put(f'range_{i + 1}', df)

8000.0 10695.0 0
10700.0 13395.0 1
13400.0 16095.0 2
16100.0 18795.0 3
18800.0 21495.0 4
21500.0 24195.0 5
24200.0 26895.0 6
26900.0 29595.0 7
29600.0 32295.0 8
32300.0 34995.0 9
35000.0 35000.0 10


In [29]:
number_of_groups = 10

with pd.HDFStore(output_path,  mode='r') as newstore:
    for data_lbl in newstore.keys():
        df_restored = newstore.select(data_lbl)
        
        step = df_restored.point_idx.nunique() //  number_of_groups
        points_arr = df_restored.point_idx.unique()
        
        range_lbl = data_lbl.replace("/", "")
        
        for i in range(number_of_groups):
            df: pd.DataFrame = df_restored.loc[(df_restored.point_idx >= points_arr[i * step]) & (df_restored.point_idx <= points_arr[(i + 1) * step - 1])].copy()
            # print(df.point_idx.min(), df.point_idx.max(), i)
            df.to_hdf(output_path.parent / f"{range_lbl}.group_{i}.hdf", key="data")
            
        if points_arr[(i + 1) * step - 1] != points_arr[-1]:
            df = original_df.loc[(original_df.point_idx >= points_arr[(i + 1) * step]) & (original_df.point_idx <= points_arr[-1])].copy()
            # print(df.point_idx.min(), df.point_idx.max(), i + 1)
            df.to_hdf(output_path.parent / f"{range_lbl}.group_{i + 1}.hdf", key="data")

TypeError: NDFrame.to_hdf() missing 1 required positional argument: 'key'